In [3]:
from kafka import KafkaConsumer
import json
import psycopg2
from datetime import datetime

# Kafka Consumer setup
consumer = KafkaConsumer(
    'parking-spot-status',
    bootstrap_servers='localhost:9092',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

# PostgreSQL connection
conn = psycopg2.connect(
    host='localhost',
    port=5432,
    database='postgres',
    user='postgres',
    password='mysecretpassword'
)
cursor = conn.cursor()

print("Kafka → PostgreSQL 3-min Snapshot Consumer started...")

last_written_minute = None  # Used to prevent duplicate writes within the same 3-min block

for msg in consumer:
    data = msg.value
    now = datetime.now()
    current_minute = now.minute

    # Only insert once every 3-minute boundary (e.g., 0, 3, 6, 9, ...)
    if current_minute % 3 == 0 and current_minute != last_written_minute:
        last_written_minute = current_minute

        for spot in data['spots']:
            spot_id = spot['id']
            status = spot['status']

            cursor.execute("""
                INSERT INTO parking_status (spot_id, status, timestamp)
                VALUES (%s, %s, %s)
            """, (spot_id, status, now))
            conn.commit()

            print(f"[3-min Snapshot] Spot {spot_id} = {status} at {now}")


Kafka → PostgreSQL 3-min Snapshot Consumer started...
[3-min Snapshot] Spot 1 = Occupied at 2025-05-05 14:03:42.929179
[3-min Snapshot] Spot 2 = Occupied at 2025-05-05 14:03:42.929179
[3-min Snapshot] Spot 3 = Occupied at 2025-05-05 14:03:42.929179
[3-min Snapshot] Spot 1 = Occupied at 2025-05-05 14:06:04.825976
[3-min Snapshot] Spot 2 = Occupied at 2025-05-05 14:06:04.825976
[3-min Snapshot] Spot 3 = Empty at 2025-05-05 14:06:04.825976
[3-min Snapshot] Spot 1 = Occupied at 2025-05-05 14:09:03.736813
[3-min Snapshot] Spot 2 = Occupied at 2025-05-05 14:09:03.736813
[3-min Snapshot] Spot 3 = Empty at 2025-05-05 14:09:03.736813
[3-min Snapshot] Spot 1 = Occupied at 2025-05-05 14:12:00.716669
[3-min Snapshot] Spot 2 = Occupied at 2025-05-05 14:12:00.716669
[3-min Snapshot] Spot 3 = Empty at 2025-05-05 14:12:00.716669
[3-min Snapshot] Spot 1 = Occupied at 2025-05-05 14:15:02.893219
[3-min Snapshot] Spot 2 = Occupied at 2025-05-05 14:15:02.893219
[3-min Snapshot] Spot 3 = Empty at 2025-05-05

KeyboardInterrupt: 